# Using the Bernstein-Vazirani Algorithm to Cheat at Mastermind

Mastermind is a game typically played between two humans. It is played as follows:

1. The players choose between being codemaker and codebreaker
2. The codemaker picks a string of colors, typically 4 wide
3. The codebreaker picks a string of colors the same length as the codemaker's
4. The codemaker tells the codebreaker how many of the colors are correct and in the right place, correct in the wrong place, and incorrect
5. Repeat until the codebreaker finally guesses the code
6. Switch sides

The winner is the player that guesses correctly in less tries

In [1]:
import qiskit

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer

from IPython.display import clear_output

First, encode the game in terms of binary numbers to make it easier to analyze for the computer.
lets say we have two different colors we can choose from, black and white. Black is 1, white is 0

In [2]:
black = '1'
white = '0'

Now we create a codemaker class and allow it to set the code and retrieve the code

In [3]:
class codemaker():
    def __init__(self):
        self.code = ''
    
    def setCode(self, code):
        self.code = ''
        for i in range(len(code)):
            self.code = self.code + code[i]
        
    def getCode(self):
        return self.code

To ensure the codemaker isn't cheating, we will give the the string to an encoder. The encoder will be the one giving the hints, so we can modify it to give different hints and change the rules of the game, as well as encode the data in a way that allows our quantum computer to play as well

The rules we will use for hints are: if the string is correct, say it is correct. otherwise, tell if the number of correct black spots are even or odd (0 counts as even)

In [4]:
class QuantumEncoder():
    def __init__(self):
        self.code = ''
    def encode(self, code):
        self.code = code
        
        length = len(code)
        qc = QuantumCircuit(length+1, length)
        
        #every correct black spot flips the extra qubit, reading the last qubit tells odd or even
        for i, value in enumerate(reversed(code)):
            if value == '1':
                qc.cx(i, len(code))
                
        return qc
    
    def check(self, input):
        odd = False
        #flip for every black spot
        for i, value in enumerate(self.code):
            if value == '1' and input[i] == '1':
                odd = !odd
              
        #return the hint and if the guess is true
        return odd, self.code == input

In [5]:
#use the bernstein-vazirani algorithm to get the correct answer
class QuantumGuesser():
    def __init__(self, backend, shots):
        self.backend = backend
        self.shots = shots
        
    def guess(self, code): #code is the encoded circuit, not the actual code
        qc = QuantumCircuit(code.num_qubits, code.num_clbits)
        
        #flip the hint bit
        qc.x(code.num_qubits - 1)
        
        #place the qubits in equal superposition
        qc.h(range(code.num_qubits))
        qc.barrier()
        
        #hook up to the encoder network to retrieve your hint
        qc = qc.compose(code)
        qc.barrier()
        
        #resolve the superposition 
        qc.h(range(code.num_qubits))
        qc.barrier()
        
        #print(qc)
        
        #measure
        qc.measure(range(code.num_qubits - 1), range(code.num_clbits))
        
        return execute(qc, backend = self.backend, shots = self.shots).result().get_counts().most_frequent()

Now we actually play the game. This will be a human vs a quantum computer. In practice, the human should do much worse than the quantum computer at guessing.

In [6]:
#have the human set the code
#bit of inheritance
class HumanSetter(codemaker):
    def __init__(self):
        super().__init__()
    def setCode(self, length):

        code = []
        for i in range(length):
            code.append(input('0/1? '))
#removed the colors thing to make the game more generalizable at the end
#        for i in range(length):
#            if code[i] == 'b':
#                code[i] = black
#            else:
#                code[i] = white
        
        super().setCode(code)
        clear_output()

In [7]:
#have the human set the code
p1 = HumanSetter()
p1.setCode(length = int(input('how long of a string? (numerical value)')))

In [8]:
#encode the data so the quantum computer can play
#a fun side effect from using python is that we get rid of the original 
encoder = QuantumEncoder()

code = encoder.encode(p1.getCode())

In [9]:
#we can print the code just to show how it is encoded
input('press enter to show')
print(code)
input('press enter to stop showing')
clear_output()

In [10]:
#guess and adjust based on the hints
backend = BasicAer.get_backend('qasm_simulator')
p2 = QuantumGuesser(backend, 1)

attempts = 1

print(f'attempt: {attempts}')
guess = p2.guess(code)

hint, correct = encoder.check(guess)

while (not correct):
    print('incorrect guess')
    print(f'number of correct black guesses is {"odd" if hint else "even"}')
    
    attempts = attempts + 1
    print(f'attempt: {attempts}')
    guess = p2.guess(code)
    
    hint, correct = encoder.check(guess)
    
print(f'you found it, the code was: {p1.getCode()}')

attempt: 1
you found it, the code was: 1010


For fun, the quantum computer can also play the role of the codemaster

In [13]:
class QuantumSetter(codemaker):
    def __init__(self, backend, shots):
        super().__init__()
        self.backend = backend
        self.shots = shots
        
    def setCode(self, length):
        qc = QuantumCircuit(length, length)
        
        qc.h(range(length))
        qc.measure(range(length), range(length))
        self.code = execute(qc, backend = self.backend, shots = self.shots).result().get_counts().most_frequent()

In [18]:
#testing
p3 = QuantumSetter(backend, 1)
p3.setCode(4)
print(p3.getCode())

1011


It works, quantum random number generators are actually easier to work with, you don't need to seed them or think of an algorithm, just apply an H gate

In [19]:
#Define a human player so we can switch sides
class HumanGuesser():
    def guess(self, code):
        length = code.num_clbits
        guessLen = 0
        
        #check that the length of the guess is correct so something doesn't break later
        while (guessLen != length):
            humanGuess = input(f'Make a guess (len:{length})')
            guessLen = len(humanGuess)
            
        return humanGuess

All the pieces are in place, now we can define the game as a class

This is easier if the human guesser is guessing a string of 1's and 0's, so while setting is with b/w, guessing is actually with 0/1

In [22]:
class Game():
    def __init__(self, setter, guesser, encoder):
        self.p1 = setter
        self.p2 = guesser
        self.encoder = encoder
        
    def play(self):
        #set the code
        length = int(input('how long of a string? (numerical value): '))
        self.p1.setCode(length)
        
        #encode the code with the rules defined by the encoder
        code = self.encoder.encode(self.p1.getCode())
        
        #track how many times it takes to guess the code
        attempts = 1

        print(f'attempt: {attempts}')
        guess = self.p2.guess(code)
    
        hint, correct = self.encoder.check(guess)

        while (not correct):
            print('incorrect guess')
            print(f'number of correct black guesses is {"odd" if hint else "even"}\n')
    
            attempts = attempts + 1
            print(f'attempt: {attempts}')
            guess = self.p2.guess(code)
    
            hint, correct = self.encoder.check(guess)
            
    
        print(f'you found it, the code was: {p1.getCode()}')

In [ ]:
choice = int(input('select player 1, 0 = quantum, 1 = human: '))
if (choice): 
    p1 = HumanSetter()
else:
    p1 = QuantumSetter(backend, 1)
clear_output()

choice = int(input('select player 2, 0 = quantum, 1 = human: '))
if choice:
    p2 = HumanGuesser()
else:
    p2 = QuantumGuesser(backend, 1)
clear_output()

encoder = QuantumEncoder()

mastermind = Game(p1, p2, encoder)

mastermind.play()

attempt: 1


Make a guess (len:4) 0001


incorrect guess
number of correct black guesses is odd

attempt: 2


Make a guess (len:4) 0010


incorrect guess
number of correct black guesses is odd

attempt: 3


Make a guess (len:4) 0100


incorrect guess
number of correct black guesses is odd

attempt: 4


Make a guess (len:4) 1000


incorrect guess
number of correct black guesses is odd

attempt: 5


Now to run it on real hardware

In [43]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options

In [44]:
class RealQuantumSetter(codemaker):
    def __init__(self, session):
        super().__init__()
        self.session = session
        
    def setCode(self, length):
        qc = QuantumCircuit(length, length)
        
        qc.h(range(length))
        qc.measure(range(length), range(length))
        
        sampler = Sampler(session = self.session)
        job = sampler.run(qc)
        print(f'setter id: {job.job_id()}')
        self.code = job.result().get_counts().most_frequent()

In [45]:
class RealQuantumGuesser():
    def __init__(self, session):
        self.session = session
    def guess(self, code): #code is the encoded circuit, not the actual code
        qc = QuantumCircuit(code.num_qubits, code.num_clbits)
        
        #flip the hint bit
        qc.x(code.num_qubits - 1)
        
        #place the qubits in equal superposition
        qc.h(range(code.num_qubits))
        qc.barrier()
        
        #hook up to the encoder network to retrieve your hint
        qc = qc.compose(code)
        qc.barrier()
        
        #resolve the superposition 
        qc.h(range(code.num_qubits))
        qc.barrier()
        
        #print(qc)
        
        #measure
        qc.measure(range(code.num_qubits - 1), range(code.num_clbits))
        
        sampler = Sampler(session = self.session)
        job = sampler.run(qc)
        print(f'guesser id: {job.job_id()}')
        
        return job.result().get_counts().most_frequent()

In [46]:
service = QiskitRuntimeService()
service.least_busy(simulator = False, operational=True, min_num_qubits=5)

<IBMBackend('ibmq_belem')>

Even though belem is the least busy, it takes 11 hours, I am going to use quito because it was recommended to me by a classmate

In [50]:
with Session(service=service, backend = 'ibmq_manila') as session:
    p1 = QuantumSetter(BasicAer.get_backend('qasm_simulator'), 1) #i tried the real setter, it gave basically equal everything. Makes sense
    p2 = RealQuantumGuesser(session)
    encoder = QuantumEncoder()
    
    quantumGame = Game(p1, p2, encoder)
    
    quantumGame.play()

how long of a string? (numerical value):  4


attempt: 1
guesser id: cee5755iubo7eglamao0
Traceback (most recent call last):
  Input In [50] in <cell line: 1>
    quantumGame.play()
  Input In [35] in play
    guess = self.p2.guess(code)
  Input In [45] in guess
    return job.result().get_counts().most_frequent()
AttributeError: 'SamplerResult' object has no attribute 'get_counts'

Use %tb to get the full traceback.
